In [1]:
import tensorflow as tf
from sys import getsizeof
import os
import h5py
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
SEQ_LEN = 10
X = np.load(f'AI/data/X_holistic_seq_{SEQ_LEN}.npy').astype('float32')
base_y = np.load(f'AI/data/y_holistic_seq_{SEQ_LEN}.npy').astype('float32')
y = tf.keras.utils.to_categorical(base_y).astype('float32')

In [3]:
X.shape

(3871, 10, 1662)

In [3]:
def get_size(file_path):
    size =  os.path.getsize(file_path)
    print(f'File size: {round(size/1024, 3)} KB')

def get_accuracy(model):
    loss, acc = model.evaluate(X, y, verbose=2)
    print(f'Accuracy: {round(100*acc, 2)}')

In [4]:
MODEL_PATH = 'AI/models/final_lstm_model___Date_Time_2023_06_19__13_31_39___Loss_0.998094916343689___Accuracy_0.6675257682800293.h5'
keras_model = tf.keras.saving.load_model(MODEL_PATH)
get_size(MODEL_PATH)
# get_accuracy(keras_model)

File size: 4817.531 KB


In [5]:
TF_LITE_MODEL_NAME = 'AI/models/final_model.tflite'

tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)

tflite_model = tf_lite_converter.convert()

open(TF_LITE_MODEL_NAME, 'wb').write(tflite_model)

get_size(TF_LITE_MODEL_NAME)

INFO:tensorflow:Assets written to: /var/folders/k1/_k87wym92vxg5hl9vzj9cz3h0000gn/T/tmpdl0_v3h3/assets


INFO:tensorflow:Assets written to: /var/folders/k1/_k87wym92vxg5hl9vzj9cz3h0000gn/T/tmpdl0_v3h3/assets
2023-06-19 13:40:25.218922: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-06-19 13:40:25.219156: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-06-19 13:40:25.350458: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


File size: 1706.629 KB


In [70]:
interpreter = tf.lite.Interpreter(TF_LITE_MODEL_NAME)
interpreter.get_input_details(), interpreter.get_output_details()

([{'name': 'serving_default_input_1:0',
   'index': 0,
   'shape': array([  1,  10, 132], dtype=int32),
   'shape_signature': array([ -1,  10, 132], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': array([], dtype=float32),
    'zero_points': array([], dtype=int32),
    'quantized_dimension': 0},
   'sparsity_parameters': {}}],
 [{'name': 'StatefulPartitionedCall:0',
   'index': 283,
   'shape': array([1, 8], dtype=int32),
   'shape_signature': array([-1,  8], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': array([], dtype=float32),
    'zero_points': array([], dtype=int32),
    'quantized_dimension': 0},
   'sparsity_parameters': {}}])

In [71]:
interpreter.resize_tensor_input(interpreter.get_input_details()[0]['index'], X.shape)
interpreter.resize_tensor_input(interpreter.get_output_details()[0]['index'], y.shape)
interpreter.allocate_tensors()
interpreter.get_input_details(), interpreter.get_output_details()

([{'name': 'serving_default_input_1:0',
   'index': 0,
   'shape': array([3834,   10,  132], dtype=int32),
   'shape_signature': array([ -1,  10, 132], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': array([], dtype=float32),
    'zero_points': array([], dtype=int32),
    'quantized_dimension': 0},
   'sparsity_parameters': {}}],
 [{'name': 'StatefulPartitionedCall:0',
   'index': 283,
   'shape': array([3834,    8], dtype=int32),
   'shape_signature': array([-1,  8], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': array([], dtype=float32),
    'zero_points': array([], dtype=int32),
    'quantized_dimension': 0},
   'sparsity_parameters': {}}])

In [72]:
interpreter.set_tensor(interpreter.get_input_details()[0]['index'], X)
interpreter.invoke()
predictions = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

predictions_classes = np.argmax(predictions, axis=1)
acc = accuracy_score(predictions_classes, base_y)

In [73]:
get_size(TF_LITE_MODEL_NAME)
print(f'Accuracy: {round(acc*100, 2)}')

File size: 711.844 KB
Accuracy: 73.84
